with open("labeled.txt","r") as f:
  text=f.read().lower().split("\n")


# Model 1 0~1 Negative conf   Model 2 0~1 Positive conf
X1 = []
Y1 = []
X2 = []
Y2 = []
for i in text:
  if i == "":
    continue
  o = i.split("------")
  print(o)
  if o[1] == "":
    X1.append(o[0].split(" "))
    Y1.append(0)
    X2.append(o[0].split(" "))
    Y2.append(0)

In [210]:
with open("Positive.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").replace('"',"").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(i.split(" "))
  Y1.append(0)
  X2.append(i.split(" "))
  Y2.append(1)

In [211]:
with open("Negative.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").replace('"',"").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(i.split(" "))
  Y1.append(1)
  X2.append(i.split(" "))
  Y2.append(0)

In [ ]:
np.sum(Y1)

In [212]:
word_to_token = {}
token_count = 0
tokened = []

for i in X1:
  line = []
  for word in i:
    try:
      line.append(word_to_token[word])
    except:
      token_count+=1
      word_to_token[word]=token_count
      line.append(word_to_token[word])
  tokened.append(line)
max_length = 0
for i in tokened:
  if len(i)>max_length:
    max_length=len(i)

padded = []
for i in tokened:
  i.extend([0]*(max_length+1-len(i)))
  padded.append(i)

In [214]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length+1)),
    tf.keras.layers.Embedding(
        input_dim=token_count+2,
        output_dim=64),
    tf.keras.layers.Conv1D(16, 5, activation='relu'),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    # tf.keras.layers.GRU(100, activation='relu', return_sequences=1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [216]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [ ]:
# model.fit(X, padded)
index = np.arange(0,len(X1))
np.random.shuffle(index)
padded = np.array(padded)[index]
Y1 = np.array(Y1)[index]
model.fit(padded, Y1, epochs=30, shuffle=True, validation_split=0.1)
# https://stackoverflow.com/questions/51581521/accuracy-stuck-at-50-keras
# THIS HELPED A LOT!!! A QUESTION THAT WAS ANNOYING ME FOR MORE THAN A YEAR!

In [ ]:
ans = []
for i in range(200):
  ans.append(model.predict([padded[i]]))

In [192]:
test = "we collect information you provide directly to us"
tokened_test = [word_to_token[word] for word in test.split(" ")]
tokened_test.extend([0]*(max_length+1-len(tokened_test)))
model.predict([tokened_test])


1/1 [==============================] - 0s 27ms/step


array([[0.02018598]], dtype=float32)

In [ ]:
tokened_test

In [220]:
word_to_token = {}
token_count = 0
tokened = []

for i in X2:
  line = []
  for word in i:
    try:
      line.append(word_to_token[word])
    except:
      token_count+=1
      word_to_token[word]=token_count
      line.append(word_to_token[word])
  tokened.append(line)
max_length = 0
for i in tokened:
  if len(i)>max_length:
    max_length=len(i)

padded2 = []
for i in tokened:
  i.extend([0]*(max_length+1-len(i)))
  padded2.append(i)

model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length+1)),
    tf.keras.layers.Embedding(
        input_dim=token_count+2,
        output_dim=64),
    tf.keras.layers.Conv1D(16, 5, activation='relu'),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    # tf.keras.layers.GRU(100, activation='relu', return_sequences=1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])
index = np.arange(0,len(X2))
np.random.shuffle(index)
padded = np.array(padded2)[index]
Y2 = np.array(Y2)[index]
model2.fit(padded, Y2, epochs=30, shuffle=True, validation_split=0.1)

Epoch 1/30
22/22 [==============================] - 1s 24ms/step - loss: 0.6809 - accuracy: 0.6142 - val_loss: 0.6848 - val_accuracy: 0.5714
Epoch 2/30
22/22 [==============================] - 0s 11ms/step - loss: 0.6619 - accuracy: 0.6387 - val_loss: 0.6836 - val_accuracy: 0.5714
Epoch 3/30
22/22 [==============================] - 0s 11ms/step - loss: 0.6447 - accuracy: 0.6387 - val_loss: 0.6801 - val_accuracy: 0.5714
Epoch 4/30
22/22 [==============================] - 0s 10ms/step - loss: 0.6245 - accuracy: 0.6387 - val_loss: 0.6703 - val_accuracy: 0.5714
Epoch 5/30
22/22 [==============================] - 0s 10ms/step - loss: 0.5931 - accuracy: 0.6387 - val_loss: 0.6804 - val_accuracy: 0.5714
Epoch 6/30
22/22 [==============================] - 0s 10ms/step - loss: 0.5421 - accuracy: 0.6561 - val_loss: 0.5983 - val_accuracy: 0.6364
Epoch 7/30
22/22 [==============================] - 0s 11ms/step - loss: 0.4751 - accuracy: 0.7442 - val_loss: 0.6251 - val_accuracy: 0.6623
Epoch 8/30
22

In [ ]:
ans

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tf = CountVectorizer()
X = tf.fit_transform(X1)

In [ ]:
X1